# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 65 new papers today
          22 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/21 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.01809


extracting tarball to tmp_2210.01809...

 done.


Retrieving document from  https://arxiv.org/e-print/2210.01810


...

extracting tarball to tmp_2210.02385...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.02408


extracting tarball to tmp_2210.02408...

 done.


/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2210.02408/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.01809-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.01809) | **A dense $\mathbf{0.1 M_{\rm \odot}}$ star in a 51-minute orbital period  eclipsing binary**  |
|| Kevin B. Burdge, et al. -- incl., <mark>Warren R. Brown</mark>, <mark>S. P. Littlefair</mark>, <mark>Frank J. Masci</mark>, <mark>Roger M. Smith</mark> |
|*Appeared on*| *2022-10-06*|
|*Comments*| *48 Pages, 12 figures, 2 tables, Published online by Nature on Oct 5, 2022*|
|**Abstract**| In over a thousand known cataclysmic variables (CVs), where a white dwarf is accreting from a hydrogen-rich star, only a dozen have orbital periods below 75 minutes. One way to achieve these short periods requires the donor star to have undergone substantial nuclear evolution prior to interacting with the white dwarf, and it is expected that these objects will transition to helium accretion. These transitional CVs have been proposed as progenitors of helium CVs. However, no known transitional CV is expected to reach an orbital period short enough to account for most of the helium CV population, leaving the role of this evolutionary pathway unclear. Here we report observations of ZTF J1813+4251, a 51-minute orbital period, fully eclipsing binary system consisting of a star with a temperature comparable to that of the Sun but a density 100 times greater due to its helium-rich composition, accreting onto a white dwarf. Phase-resolved spectra, multi-band light curves and the broadband spectral energy distribution allow us to obtain precise and robust constraints on the masses, radii and temperatures of both components. Evolutionary modeling shows that ZTF J1813+4251 is destined to become a helium CV binary, reaching an orbital period under 20 minutes, rendering ZTF J1813+4251 a previously missing link between helium CV binaries and hydrogen-rich CVs. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.01811-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.01811) | **Discovery of 24 radio-bright quasars at $4.9 \leq z \leq6.6$ using  low-frequency radio observations**  |
|| A. J. Gloudemans, et al. -- incl., <mark>J. F. Hennawi</mark>, <mark>D. J. B. Smith</mark> |
|*Appeared on*| *2022-10-06*|
|*Comments*| *19 pages, 11 figures, accepted for publication in A&A*|
|**Abstract**| High redshift quasars ($z>5$) that also shine brightly at radio wavelengths are unique signposts of supermassive black hole activity in the early universe. However, bright radio sources at $z\ge5$ are extremely rare and therefore we have started a campaign to search for new high-$z$ quasars by combining an optical dropout selection driven by the $g$, $r$, and $z$ bands from the Dark Energy Spectroscopic Instrument (DESI) Legacy Imaging Surveys with low-frequency radio observations from the LOFAR Two-metre Sky Survey (LoTSS). Currently, LoTSS covers a large fraction of the northern sky (5720 deg$^2$) to such a depth (median noise level of 83 $\mu$Jy beam$^{-1}$) that about 30% of the general quasar population is detected $-$ which is a factor of 5-10 more than previous large sky radio surveys such as NVSS and FIRST, respectively. In this paper, we present the discovery of 20 new quasars (and the independent confirmation of 4) between $4.9\leq z\leq 6.6$. Out of the 24 quasars, 21 satisfy the traditional radio-loudness criterion of $R=f_{5\text{GHz}}/f_{4400A} > 10$, with the full sample spanning $R\sim$6-1000, thereby more than doubling the sample of known radio-loud quasars at $z \ge 5$. Our radio detection requirement strongly decreases the contamination of stellar sources and allows one to select these quasars in a broad redshift range. Despite selecting our quasar candidates using fewer and less conservative colour restrictions, both the optical and near-infrared colours, Ly$\alpha$ emission line properties, and dust reddening, $E(B-V)$, measurements of our quasar sample do not deviate from the known radio-quiet quasar population, suggesting similar optical quasar properties of the radio-loud and radio-quiet quasar population at high-$z$. Our campaign demonstrates the potential for discovering new high-$z$ quasar populations through next generation radio continuum surveys. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.01810-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.01810) | **X-ray Reverberation Mapping of Ark 564 using Gaussian Process Regression**  |
|| Collin D. Lewin, et al. -- incl., <mark>Rachel Zhang</mark> |
|*Appeared on*| *2022-10-06*|
|*Comments*| *19 pages, 9 figures. Accepted for publication in The Astrophysics Journal*|
|**Abstract**| Ark 564 is an extreme high-Eddington Narrow-line Seyfert 1 galaxy, known for being one of the brightest, most rapidly variable soft X-ray AGN, and for having one of the lowest temperature coronae. Here we present a 410-ks NuSTAR observation and two 115-ks XMM-Newton observations of this unique source, which reveal a very strong, relativistically broadened iron line. We compute the Fourier-resolved time lags by first using Gaussian processes to interpolate the NuSTAR gaps, implementing the first employment of multi-task learning for application in AGN timing. By fitting simultaneously the time lags and the flux spectra with the relativistic reverberation model RELTRANS, we constrain the mass at $2.3^{+2.6}_{-1.3} \times 10^6M_\odot$, although additional components are required to describe the prominent soft excess in this source. These results motivate future combinations of machine learning, Fourier-resolved timing, and the development of reverberation models. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.02339-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.02339) | **Particle clustering in turbulence: Prediction of spatial and statistical  properties with deep learning**  |
|| Yan-Mong Chan, et al. -- incl., <mark>Yin Li</mark> |
|*Appeared on*| *2022-10-06*|
|*Comments*| *19 pages, 13 figures, submitted to ApJ*|
|**Abstract**| We demonstrate the utility of deep learning for modeling the clustering of particles that are aerodynamically coupled to turbulent fluids. Using a Lagrangian particle module within the ATHENA++ hydrodynamics code, we simulate the dynamics of particles in the Epstein drag regime within a periodic domain of isotropic forced hydrodynamic turbulence. This setup is an idealized model relevant to the collisional growth of micron to mmsized dust particles in early stage planet formation. The simulation data is used to train a U-Net deep learning model to predict gridded three-dimensional representations of the particle density and velocity fields, given as input the corresponding fluid fields. The trained model qualitatively captures the filamentary structure of clustered particles in a highly non-linear regime. We assess model fidelity by calculating metrics of the density structure (the radial distribution function) and of the velocity field (the relative velocity and the relative radial velocity between particles). Although trained only on the spatial fields, the model predicts these statistical quantities with errors that are typically < 10%. Our results suggest that, given appropriately expanded training data, deep learning could be used to accelerate calculations of particle clustering and collision outcomes both in protoplanetary disks, and in related two-fluid turbulence problems that arise in other disciplines. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.02363-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.02363) | **Proton Synchrotron Origin of the Very High Energy Emission of GRB  190114C**  |
|| <mark>Hebzibha Isravel</mark>, Asaf Pe'er, Damien Begue |
|*Appeared on*| *2022-10-06*|
|*Comments*| *17 pages, 12 figures*|
|**Abstract**| We consider here a proton-synchrotron model to explain the MAGIC observation of GRB 190114C afterglow in the energy band $0.2 - 1$ TeV, while the X-ray spectra are explained by electron-synchrotron emission. Given the uncertainty of the particle acceleration process, we consider several variations of the model, and show that they all match the data very well. We find that the values of the uncertain model parameters are reasonable: explosion energy $\sim 10^{54.5}$ erg, ambient density $\sim 10-100 {\rm cm^{-3}}$, and fraction of electrons/ protons accelerated to a high energy power law of a few per-cents. All these values are directly derived from the observed TeV and X-ray fluxes. They are consistent with both late time data at all bands, from radio to X-rays, and with numerical models of particle acceleration. Our results thus demonstrate the relevance of proton-synchrotron emission to the high energy observations of GRBs during their afterglow phase. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.02385-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.02385) | **Impact of inhomogeneous reionization on post-reionization 21 cm  intensity mapping measurement of cosmological parameters**  |
|| <mark>Heyang Long</mark>, Catalina Morales-Gutiérrez, <mark>Paulo Montero-Camacho</mark>, Christopher M. Hirata |
|*Appeared on*| *2022-10-06*|
|*Comments*| *12 pages, 8 figures. Comments welcome*|
|**Abstract**| 21 cm intensity mapping (IM) has the potential to be a strong and unique probe of cosmology from redshift of order unity to redshift potentially as high as 30. For post-reionization 21 cm observations, the signal is modulated by the thermal and dynamical reaction of gas in the galaxies to the passage of ionization fronts during the Epoch of Reionization. In this work, we investigate the impact of inhomogeneous reionization on the post-reionization 21 cm power spectrum and the induced shifts of cosmological parameters at redshifts $3.5 \lesssim z \lesssim 5.5$. We make use of hydrodynamics simulations that could resolve small-scale baryonic structure evolution to quantify HI abundance fluctuation, while semi-numerical large box 21cmFAST simulations capable of displaying inhomogeneous reionization process are deployed to track the inhomogeneous evolution of reionization bubbles. We find the inhomogeneous reionization effect could impact the HI power spectrum up to tens of percent level and shift cosmological parameters estimation from sub-percent to tens percent in the observation of future post-reionization 21 cm intensity mapping experiments SKA-LOW and PUMA. In particular, the shift is up to 0.033 in the spectral index $n_s$ and 0.025 eV in the sum of the neutrino masses $\sum m_\nu$ depending on the reionization model and the observational parameters. We discuss strategies to mitigate and separate these biases. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.02408-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.02408) | **Bayesian statistics approach to imaging of aperture synthesis data:  RESOLVE meets ALMA**  |
|| Łukasz Tychoniec, et al. -- incl., <mark>Eric Villard</mark> |
|*Appeared on*| *2022-10-06*|
|*Comments*| *Proceedings of International Workshop on Bayesian Inference and Maximum Entropy Methods in Science and Engineering, IHP, Paris, July 18-22, 2022*|
|**Abstract**| The Atacama Large Millimeter/submillimeter Array (ALMA) is currently revolutionizing observational astrophysics. The aperture synthesis technique provides angular resolution otherwise unachievable with the conventional single-aperture telescope. However, recovering the image from the inherently undersampled data is a challenging task. The CLEAN algorithm has proven successful and reliable and is commonly used in imaging the interferometric observations. It is not, however, free of limitations. Point-source assumption, central to the CLEAN is not optimal for the extended structures of molecular gas recovered by ALMA. Additionally, negative fluxes recovered with CLEAN are not physical. This begs to search for alternatives that would be better suited for specific science cases. We present the recent developments in imaging ALMA data using Bayesian inference techniques, namely the RESOLVE algorithm This algorithm, based on information field theory \cite{Ensslin2013}, has been already successfully applied to image the Very Large Array data. We compare the capability of both CLEAN and RESOLVE to recover known sky signal, convoluted with the simulator of ALMA observation data and we investigate the problem with a set of actual ALMA observations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.02276-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.02276) | **CASA, the Common Astronomy Software Applications for Radio Astronomy**  |
|| CASA TEAM, et al. -- incl., <mark>Jennifer Donovan Meyer</mark>, <mark>Pamela Harris</mark>, <mark>Joseph Masters</mark> |
|*Appeared on*| *2022-10-06*|
|*Comments*| *Accepted for publication in PASP (20 pages, 4 figures). Joint publication with CASA-VLBI paper*|
|**Abstract**| CASA, the Common Astronomy Software Applications, is the primary data processing software for the Atacama Large Millimeter/submillimeter Array (ALMA) and the Karl G. Jansky Very Large Array (VLA), and is frequently used also for other radio telescopes. The CASA software can handle data from single-dish, aperture-synthesis, and Very Long Baseline Interferometery (VLBI) telescopes. One of its core functionalities is to support the calibration and imaging pipelines for ALMA, VLA, VLA Sky Survey (VLASS), and the Nobeyama 45m telescope. This paper presents a high-level overview of the basic structure of the CASA software, as well as procedures for calibrating and imaging astronomical radio data in CASA. CASA is being developed by an international consortium of scientists and software engineers based at the National Radio Astronomical Observatory (NRAO), the European Southern Observatory (ESO), the National Astronomical Observatory of Japan (NAOJ), and the Joint Institute for VLBI European Research Infrastructure Consortium (JIV-ERIC), under the guidance of NRAO. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2210.01809.md
    + _build/html/tmp_2210.01809/./Figures/ZTF_LC_Red.png
    + _build/html/tmp_2210.01809/./Figures/HiPERCAM_LC_v2.png
    + _build/html/tmp_2210.01809/./Figures/Good_RV.png
exported in  _build/html/2210.01811.md
    + _build/html/tmp_2210.01811/./Fig5_spectral_index.png
    + _build/html/tmp_2210.01811/./Fig2_spectra_abs_flux_corr.png
    + _build/html/tmp_2210.01811/./Fig10_e_b_v_distribution.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\bibinfo}[2]{#2}$
$\newcommand{\eprint}[2][]{\url{#2}}$
$\newcommand{\bibinfo}[2]{#2}$
$\newcommand{\eprint}[2][]{\url{#2}}$
$\newcommand{\mn}{{Mon. Not. R. Astron. Soc.}}$
$\newcommand{\mnras}{\mn}$
$\newcommand{\aj}{{"Astron. J."}}$
$\newcommand{\apj}{{Astrophys. J.}}$
$\newcommand{\apjl}{{Astrophys. J. Lett.}}$
$\newcommand{\apjs}{{Astrophys. J. Supp.}}$
$\newcommand{\apss}{{Astrophys. and Space Sc.}}$
$\newcommand{\aaps}{{Astron. Astrophys. Supp.}}$
$\newcommand{\aap}{{Astron. Astrophys.}}$
$\newcommand{\nat}{{Nature}}$
$\newcommand{\pasj}{{PASJ}}$
$\newcommand{\prd}{{Phys. Rev. D}}$
$\newcommand{\prl}{{Phys. Rev. Lett.}}$
$\newcommand{\pasp}{{Pub. Ast. Soc. Pac.}}$
$\newcommand{\procspie}{Proc. SPIE}$
$\newcommand{\ssr}{Space Science Reviews}$
$\newcommand{\araa}{Annual Review of Astronomy and Astrophysics}$
$\newcommand{\thetable}{\arabic{table}}$
$\newcommand{\figurename}{Extended Data Figure}$
$\newcommand{\figurename}{Extended Data Figure}$
$\newcommand{\figurename}{Extended Data Figure}$
$\newcommand{\figurename}{Extended Data Figure}$
$\newcommand{\figurename}{Extended Data Figure}$
$\newcommand{\figurename}{Extended Data Figure}$
$\newcommand{\figurename}{Extended Data Figure}$
$\newcommand{\figurename}{Extended Data Figure}$
$\newcommand{\}{url}$
$\newcommand{\urlprefix}{URL }$
$\newcommand{\}{url}$
$\newcommand{\urlprefix}{URL }$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\bibinfo$}[2]{#2}$
$\newcommand{$\eprint$}[2][]{\url{#2}}$
$\newcommand{$\bibinfo$}[2]{#2}$
$\newcommand{$\eprint$}[2][]{\url{#2}}$
$\newcommand{$\mn$}{{Mon. Not. R. Astron. Soc.}}$
$\newcommand{$\mn$ras}{$\mn$}$
$\newcommand{$\aj$}{{"Astron. J."}}$
$\newcommand{$\apj$}{{Astrophys. J.}}$
$\newcommand{$\apj$l}{{Astrophys. J. Lett.}}$
$\newcommand{$\apj$s}{{Astrophys. J. Supp.}}$
$\newcommand{$\apss$}{{Astrophys. and Space Sc.}}$
$\newcommand{$\aaps$}{{Astron. Astrophys. Supp.}}$
$\newcommand{$\aap$}{{Astron. Astrophys.}}$
$\newcommand{$\nat$}{{Nature}}$
$\newcommand{$\pasj$}{{PASJ}}$
$\newcommand{$\prd$}{{Phys. Rev. D}}$
$\newcommand{$\prl$}{{Phys. Rev. Lett.}}$
$\newcommand{$\pasp$}{{Pub. Ast. Soc. Pac.}}$
$\newcommand{$\procspie$}{Proc. SPIE}$
$\newcommand{$\ssr$}{Space Science Reviews}$
$\newcommand{$\araa$}{Annual Review of Astronomy and Astrophysics}$
$\newcommand{$\thetable$}{\arabic{table}}$
$\newcommand{$\figurename$}{Extended Data Figure}$
$\newcommand{$\figurename$}{Extended Data Figure}$
$\newcommand{$\figurename$}{Extended Data Figure}$
$\newcommand{$\figurename$}{Extended Data Figure}$
$\newcommand{$\figurename$}{Extended Data Figure}$
$\newcommand{$\figurename$}{Extended Data Figure}$
$\newcommand{$\figurename$}{Extended Data Figure}$
$\newcommand{$\figurename$}{Extended Data Figure}$
$\newcommand{\}{url}$
$\newcommand{$\urlprefix$}{URL }$
$\newcommand{\}{url}$
$\newcommand{$\urlprefix$}{URL }$</div>



<div id="title">

#  star in a 51-minute orbital period eclipsing binary

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.01809-b31b1b.svg)](https://arxiv.org/abs/2210.01809)<mark>Appeared on: 2022-10-06</mark> - _48 Pages, 12 figures, 2 tables, Published online by Nature on Oct 5, 2022_

</div>
<div id="authors">

Kevin B. Burdge, et al. -- incl., <mark><mark>Warren R. Brown</mark></mark>, <mark><mark>S. P. Littlefair</mark></mark>, <mark><mark>Frank J. Masci</mark></mark>, <mark><mark>Roger M. Smith</mark></mark>

</div>
<div id="abstract">

**Abstract:** In over a thousand known cataclysmic variables (CVs), where a white dwarf is accreting from a hydrogen-rich star, only a dozen have orbital periods below 75 minutes\cite{Green2020,Kato2009,Littlefield2013,Ramsay2014,Augusteijn1996,Kato2014,Thorstensen2002,Breedt2012,Kato2015}. One way to achieve these short periods requires the donor star to have undergone substantial nuclear evolution prior to interacting with the white dwarf\cite{Sienkiewicz1984,Rappaport1984,Nelson1986,Kalomeni2016,Podsiadlowski2003}, and it is expected that these objects will transition to helium accretion. These transitional CVs have been proposed as progenitors of helium CVs\cite{Kalomeni2016,Podsiadlowski2003,Nelemans2010,Goliasch2015,Ramsay2018,El-Badry2021}. However, no known transitional CV is expected to reach an orbital period short enough to account for most of the helium CV population, leaving the role of this evolutionary pathway unclear. Here we report observations of ZTF J1813+4251, a 51-minute orbital period, fully eclipsing binary system consisting of a star with a temperature comparable to that of the Sun but a density 100 times greater due to its helium-rich composition, accreting onto a white dwarf. Phase-resolved spectra, multi-band light curves and the broadband spectral energy distribution allow us to obtain precise and robust constraints on the masses, radii and temperatures of both components. Evolutionary modeling shows that ZTF J1813+4251 is destined to become a helium CV binary, reaching an orbital period under 20 minutes, rendering ZTF J1813+4251 a previously missing link between helium CV binaries and hydrogen-rich CVs.

</div>

<div id="div_fig1">

<img src="tmp_2210.01809/./Figures/ZTF_LC_Red.png" alt="Fig3" width="100%"/>

**Figure 3. -** \textbf{a)} The ZTF light curve of ZTF J1813+4251. The system has been in quiescence in all ZTF observations, except for those taken on September 21, 2019, in which the system brightened by over a factor of two. \textbf{b)} A zoom in of the ZTF data taken on September 21, 2019, during the outburst. The final of the four measurements during the outburst is significantly fainter because it occurred during the primary eclipse of the system. \textbf{c)} The phase-folded ZTF data, illustrating that the faint point which occurred during the outburst coincided with the primary eclipse in the system. \textbf{d)} The red side of the Keck LRIS spectrum, coadded in the rest frame of the accreting white dwarf. Double-peaked emission lines associated with the blueshifted and redshifted components of an accretion disk are clearly visible. This disk exhibits emission associated with several elements, including hydrogen, helium and calcium. \textbf{e)} The profile of the hydrogen H$\alpha$ emission feature, the strongest disk feature in the optical spectrum. (*fig:Red_Spectrum*)

</div>
<div id="div_fig2">

<img src="tmp_2210.01809/./Figures/HiPERCAM_LC_v2.png" alt="Fig1" width="100%"/>

**Figure 1. -** \textbf{a)} The five color HiPERCAM light curve of ZTF J1813+4251, with filters $u_{\mathrm{s}}$, $g_{\mathrm{s}}$, $r_{\mathrm{s}}$, $i_{\mathrm{s}}$ and $z_{\mathrm{s}}$ arranged from top to bottom. The object exhibits a strong sinusoidal component at twice the orbital frequency due to the tidal deformation of the donor star. The system undergoes a full eclipse, in which the donor fully occults the accreting white dwarf. The depth of the eclipse varies dramatically with wavelength, with the white dwarf contributing half the luminosity in the $u_{\mathrm{s}}$ filter, while only about ten percent in the $z_{\mathrm{s}}$ filter, primarily because the white dwarf preferentially emits its radiation at shorter wavelengths than the cooler donor. \textbf{b)} Best fit light curve models of the primary and secondary eclipses of ZTF J1813+4251. These models, combined with the spectroscopically derived radial velocity semi-amplitude, allow us to robustly constrain the properties of both components in the system. (*fig:LC*)

</div>
<div id="div_fig3">

<img src="tmp_2210.01809/./Figures/Good_RV.png" alt="Fig2" width="100%"/>

**Figure 2. -** \textbf{a)} A sinusoidal fit to the measured radial velocities of the donor star in ZTF J1813+4251, with a best-fit velocity semi-amplitude of $K_2=461.3\pm3.4\rm   km  s^{-1}$ and systemic velocity of $\gamma=-36.8\pm2.4\rm   km  s^{-1}$. \textbf{b)} The blue side of the Keck LRIS spectrum, coadded in the rest frame of the donor star. The spectrum exhibits a large number of narrow metal lines, including those from calcium, iron and magnesium, characteristic of main sequence stars with spectral types similar to the Sun. \textbf{c)} The trailed blue Keck LRIS spectra, illustrating the significant wavelength shifts in the spectrum caused by the large Doppler shifts of the donor. The trailed spectra reveal these Doppler shifts occur in the large number of narrow lines associated with the donor spectrum. (*fig:Spectrum*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\RNum}[1]{\uppercase\expandafter{\romannumeral #1\relax}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\RNum$}[1]{\uppercase\expandafter{\romannumeral #1\relax}}$</div>



<div id="title">

# Based on observations obtained with the Hobby-Eberly Telescope (HET), which is a joint project of the University of Texas at Austin, Pennsylvania State University, Ludwig-Maximillians-Universitaet Muenchen, and Georg-August Universitaet Goettingen. The HET is named in honor of its principal benefactors, William P. Hobby and Robert E. Eberly.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.01811-b31b1b.svg)](https://arxiv.org/abs/2210.01811)<mark>Appeared on: 2022-10-06</mark> - _19 pages, 11 figures, accepted for publication in A&A_

</div>
<div id="authors">

A. J. Gloudemans, et al. -- incl., <mark><mark>J. F. Hennawi</mark></mark>, <mark><mark>D. J. B. Smith</mark></mark>

</div>
<div id="abstract">

**Abstract:** High redshift quasars ($z>5$) that also shine brightly at radio wavelengths are unique signposts of supermassive black hole activity in the early universe. However, bright radio sources at$z \ge 5$are extremely rare and therefore we have started a campaign to search for new high-$z$quasars by combining an optical dropout selection driven by the$g$,$r$, and$z$bands from the Dark Energy Spectroscopic Instrument (DESI) Legacy Imaging Surveys with low-frequency radio observations from the LOFAR Two-metre Sky Survey (LoTSS). Currently, LoTSS covers a large fraction of the northern sky ($\sim$5720 deg$^2$) to such a depth (median noise level$\sim83$$\mu$Jy beam$^{-1}$) that about 30\%of the general quasar population is detected$-$which is a factor of 5-10 more than previous large sky radio surveys such as NVSS and FIRST, respectively. In this paper, we present the discovery of 20 new quasars (and the independent confirmation of four) between$4.9 \leq z \leq6.6$. Out of the 24 quasars, 21 satisfy the traditional radio-loudness criterion of$R=f_{5\text{GHz}}/f_{4400 \text{\AA}} > 10$, with the full sample spanning$R\sim$6-1000, thereby more than doubling the sample of known radio-loud quasars at$z \ge 5$. Our radio detection requirement strongly decreases the contamination of stellar sources and allows one to select these quasars in a broad redshift range. Despite selecting our quasar candidates using fewer and less conservative colour restrictions, both the optical and near-infrared colours, Ly$\alpha$emission line properties, and dust reddening,$E(B-V)$, measurements of our quasar sample do not deviate from the known radio-quiet quasar population, suggesting similar optical quasar properties of the radio-loud and radio-quiet quasar population at high-$z$. Our campaign demonstrates the potential for discovering new high-$z$quasar populations through next generation radio continuum surveys.

</div>

<div id="div_fig1">

<img src="tmp_2210.01811/./Fig5_spectral_index.png" alt="Fig3" width="100%"/>

**Figure 3. -** Radio spectral index between 144 MHz (LoTSS-DR2) and 1.4 GHz (VLA FIRST) for the discovered quasars in the FIRST and VLASS survey area. Upper limits (indicated by downward-pointing triangles) are given for the quasars non-detected in FIRST. The dashed grey line shows the stacked spectral index obtained by \cite{Gloudemans2021A&A...656A.137G} using a sample of 115 known quasars at $z>5$. ILTJ2201+2338 is outside of the FIRST footprint, but is detected in VLASS at 2-4 GHz, which enables a spectral index measurement. ILTJ1523+2935 is only marginally detected in FIRST (S/N$\sim$2), but is more securely detected in VLASS (S/N$=3.4$). (*fig:radio_spectral_index*)

</div>
<div id="div_fig2">

<img src="tmp_2210.01811/./Fig2_spectra_abs_flux_corr.png" alt="Fig2" width="100%"/>

**Figure 2. -** Absolute flux scale correction factors between the quasar spectra and PS1 $i$-band and Legacy $z$-band photometry. A correction factor of $>1$ means the spectrum continuum is lower than the photometric measurement. These corrections do not affect the redshift and Ly$\alpha$ EW estimates, but do decrease or increase the measured Ly$\alpha$ emission line flux. (*fig:abs_flux_scale*)

</div>
<div id="div_fig3">

<img src="tmp_2210.01811/./Fig10_e_b_v_distribution.png" alt="Fig5" width="100%"/>

**Figure 5. -** Distribution of $E(B-V)$ for the discovered quasars in this work and the literature sample determined by fitting the composite quasars spectrum of \cite{Selsing2016A&A...585A..87S} together with the SMC extinction law \citep{Pei1992ApJ...395..130P}. The radio-loud quasars from the literature sample are stacked on top of quasars from this work. We do not find a significant deviation between the E(B-V) distribution of the RL and RQ quasar sample. For comparison we also plot the $E(B-V)$ Gaussian distribution found by \cite{Glikman2022ApJ...934..119G} of SDSS QSOs at $0.1 < z \lesssim 3$ using a similar template fitting method.  (*fig:e_b_v_plot*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

57  publications in the last 7 days.
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers